In [1]:
##Libraries
#Dataframe & Arrays
import pandas as pd
import numpy as np



In [2]:
df = pd.read_csv('../data/dataframe_awp_staging.csv')

In [3]:
df = df.sort_values(by=['StockCode','UnitPrice'], ascending=False)

In [28]:
df_pricing_file = df.groupby(['B2B','StockCode','UnitPrice']).agg({'Rev': ['sum','count'],'Quantity':['sum']}).reset_index()
df_pricing_file = df_pricing_file.sort_values(by=['StockCode','UnitPrice'], ascending=False)
df_pricing_file.head()

B2B StockCode UnitPrice    Rev       Quantity
                                 sum count      sum
9105  1.0    90214Z      0.29   3.48     1       12
9104  1.0    90214Y      1.25   1.25     1        1
9103  1.0    90214Y      0.29  17.40     2       60
9102  1.0    90214W      0.29   3.48     1       12
9101  1.0    90214V      1.25   2.50     2        2

In [29]:
df_pricing_file.B2B = df_pricing_file.B2B.astype(int)

In [30]:
df_pricing_file.columns = ['B2B','StockCode','UnitPrice','Rev','Count','Qty']
df_pricing_file.head()

,B2B,StockCode,UnitPrice,Rev,Count,Qty
9105,1,90214Z,0.29,3.48,1,12
9104,1,90214Y,1.25,1.25,1,1
9103,1,90214Y,0.29,17.40,2,60
9102,1,90214W,0.29,3.48,1,12
9101,1,90214V,1.25,2.50,2,2


In [31]:
df_awp = df_pricing_file.groupby(['B2B','StockCode']).agg({'Rev': ['sum'],'Qty':['sum']})
df_awp.columns = ['Rev','Qty']

In [32]:
df_awp.head()

Rev  Qty
B2B StockCode             
0   11001        1.69    1
    15036      369.96  612
    15039       18.70   22
    15044A       5.90    2
    15044B       8.85    3

In [33]:
def AWP(row):
    return (row['Rev'] / row['Qty'])

In [34]:
df_awp['AWP'] = (df_awp['Rev'] / df_awp['Qty'])
df_awp.head()

Rev  Qty      AWP
B2B StockCode                      
0   11001        1.69    1  1.69000
    15036      369.96  612  0.60451
    15039       18.70   22  0.85000
    15044A       5.90    2  2.95000
    15044B       8.85    3  2.95000

In [35]:
df_awp.reset_index(inplace=True)
df_awp.columns  = ['B2B','StockCode','Rev','Qty','AWP']
df_awp.head()

,B2B,StockCode,Rev,Qty,AWP
0,0,11001,1.69,1,1.69000
1,0,15036,369.96,612,0.60451
2,0,15039,18.70,22,0.85000
3,0,15044A,5.90,2,2.95000
4,0,15044B,8.85,3,2.95000


In [36]:
## trying to identify average  Qty purchased by StockCode and customer type

In [37]:
df_qty_mean = df.groupby(['B2B','StockCode'])['Quantity'].mean().astype(int).to_frame()
df_qty_mean.reset_index(inplace=True)
df_qty_mean.columns  = ['B2B','StockCode','Quantity']

In [38]:
df_qty_mean.head()

,B2B,StockCode,Quantity
0,0.0,11001,1
1,0.0,15036,153
2,0.0,15039,11
3,0.0,15044A,2
4,0.0,15044B,1


In [39]:
def qtymeanlookup(row):
    """
    Input: StockCode taken from row within dataframe
    Output: The corresponding mean quantity sold for that item
    
    The purpose of this funtion is to vlookup value of item from a reference dataframe (df_qty_mean)
    """
    try:

        return int(df_qty_mean.loc[(df_qty_mean['StockCode'] == row.StockCode) & (df_qty_mean['B2B'] == row.B2B)]["Quantity"])

    except:

        pass

In [40]:
df_awp['MeanQty'] = df_awp.apply(qtymeanlookup, axis=1)

In [41]:
df_awp.head()

,B2B,StockCode,Rev,Qty,AWP,MeanQty
0,0,11001,1.69,1,1.69000,1
1,0,15036,369.96,612,0.60451,153
2,0,15039,18.70,22,0.85000,11
3,0,15044A,5.90,2,2.95000,2
4,0,15044B,8.85,3,2.95000,1


In [20]:
df_awp.to_csv('../data/df_awp.csv', index=False)

In [ ]:
#####################################################
#####################################################
# Create - a different pricing file without B2B Flag
#####################################################
#####################################################

In [42]:
df_pricing_file_noB2B = df.groupby(['StockCode','UnitPrice']).agg({'Rev': ['sum','count'],'Quantity':['sum']}).reset_index()
df_pricing_file_noB2B = df_pricing_file_noB2B.sort_values(by=['StockCode','UnitPrice'], ascending=False)
df_pricing_file_noB2B.head()

StockCode UnitPrice    Rev       Quantity
                            sum count      sum
8055    90214Z      0.29   3.48     1       12
8054    90214Y      1.25   1.25     1        1
8053    90214Y      0.29  17.40     2       60
8052    90214W      0.29   3.48     1       12
8051    90214V      1.25   2.50     2        2

In [43]:
df_pricing_file_noB2B.columns = ['StockCode','UnitPrice','Rev','Count','Qty']
df_pricing_file_noB2B.head()

,StockCode,UnitPrice,Rev,Count,Qty
8055,90214Z,0.29,3.48,1,12
8054,90214Y,1.25,1.25,1,1
8053,90214Y,0.29,17.40,2,60
8052,90214W,0.29,3.48,1,12
8051,90214V,1.25,2.50,2,2


In [45]:
df_awp_noB2B = df_pricing_file_noB2B.groupby(['StockCode']).agg({'Rev': ['sum'],'Qty':['sum']})
df_awp_noB2B.columns = ['Rev','Qty']

In [46]:
df_awp_noB2B.head()

,Rev,Qty
StockCode,,
10002,699.55,823
10080,114.41,291
10120,40.53,193
10123C,3.25,5
10124A,6.72,16


In [47]:
df_awp_noB2B['AWP'] = (df_awp_noB2B['Rev'] / df_awp_noB2B['Qty'])
df_awp_noB2B.head()

,Rev,Qty,AWP
StockCode,,,
10002,699.55,823,0.850000
10080,114.41,291,0.393162
10120,40.53,193,0.210000
10123C,3.25,5,0.650000
10124A,6.72,16,0.420000


In [48]:
df_awp_noB2B.reset_index(inplace=True)
df_awp_noB2B.columns  = ['StockCode','Rev','Qty','AWP']
df_awp_noB2B.head()

,StockCode,Rev,Qty,AWP
0,10002,699.55,823,0.850000
1,10080,114.41,291,0.393162
2,10120,40.53,193,0.210000
3,10123C,3.25,5,0.650000
4,10124A,6.72,16,0.420000


In [49]:
df_qty_mean_noB2B = df.groupby(['StockCode'])['Quantity'].mean().astype(int).to_frame()
df_qty_mean_noB2B.reset_index(inplace=True)
df_qty_mean_noB2B.columns  = ['StockCode','Quantity']

In [50]:
df_qty_mean_noB2B.head()

,StockCode,Quantity
0,10002,16
1,10080,13
2,10120,6
3,10123C,1
4,10124A,3


In [51]:
def qtymeanlookup_noB2B(row):
    """
    Input: StockCode taken from row within dataframe
    Output: The corresponding mean quantity sold for that item
    
    The purpose of this funtion is to vlookup value of item from a reference dataframe (df_qty_mean)
    """
    try:

        return int(df_qty_mean_noB2B.loc[(df_qty_mean_noB2B['StockCode'] == row.StockCode) ]["Quantity"])

    except:

        pass

In [52]:
df_awp_noB2B['MeanQty'] = df_awp_noB2B.apply(qtymeanlookup_noB2B, axis=1)

In [53]:
df_awp_noB2B.head()

,StockCode,Rev,Qty,AWP,MeanQty
0,10002,699.55,823,0.850000,16
1,10080,114.41,291,0.393162,13
2,10120,40.53,193,0.210000,6
3,10123C,3.25,5,0.650000,1
4,10124A,6.72,16,0.420000,3


In [54]:
df_awp_noB2B.to_csv('../data/df_awp_noB2B.csv', index=False)